# Parameterized Decorators

In [7]:
from functools import wraps

def say(text):
    def _say(func):
        @wraps(func)
        def __say(*args, **kwargs):
            print(text)
            return func(*args, **kwargs)
        return __say
    return _say

In [8]:
@say('Hello')
def add(a, b):
    return a + b

In [9]:
add(1, 2)

Hello


3

In [10]:
@say('Hello')
@say('Goodbye')
def add(a, b):
    return a + b

In [11]:
add(1, 2)

Hello
Goodbye


3

In [12]:
sorted?


Signature: sorted(iterable, /, *, key=None, reverse=False)
Docstring:
Return a new list containing all items from the iterable in ascending order.

A custom key function can be supplied to customize the sort order, and the
reverse flag can be set to request the result in descending order.
Type:      builtin_function_or_method

In [13]:
callable(add)

True

In [14]:
callable(say)

True

In [15]:
class CallCounter():
    def __init__(self):
        self.count = 0
    
    def __call__(self): # this makes the class callable
        self.count += 1

In [16]:
c = CallCounter()

In [17]:
c.count

0

In [18]:
c.count

0

In [19]:
c()

In [20]:
c.count

1

In [21]:
class Say:
    def __init__(self, text):
        self.text = text

    def __call__(self, func):
            @wraps(func)
            def say(*args, **kwargs):
                print(self.text)
                return func(*args, **kwargs)
            return say

In [22]:
@Say('Hello')
def add(a, b):
    return a + b

In [23]:
add(1, 2)

Hello


3

In [24]:
isinstance(1, int)

True

In [25]:
isinstance(True, int)

True

In [26]:
bool.__mro__

(bool, int, object)

In [27]:
True + 1

2

In [28]:
False + 1

1

In [44]:
from functools import wraps

In [45]:
def check(*argtypes):
    """Check function argument types."""

    def _check(func):
        """Take function as argument."""

        @wraps(func)
        def __check(*args):
            """Take function arguments."""
            if len(args) != len(argtypes):
                msg = f'Expected {len(argtypes)} but got {len(args)} arguments'
                raise TypeError(msg)
            for arg, argtype in zip(args, argtypes):
                if not isinstance(arg, argtype):
                    msg = f'Expected {argtypes} but got '
                    msg += f'{tuple(type(arg) for arg in args)}'
                    raise TypeError(msg)
            return func(*args)

        return __check

    return _check


In [46]:
@check(float, float)
def add(a, b):
    return a + b

In [47]:
add(1.0, 2.0)

3.0

In [48]:
dict.setdefault?

Signature: dict.setdefault(self, key, default=None, /)
Docstring:
Insert key with a value of default if key is not in the dictionary.

Return the value for key if key is in the dictionary, else default.
Type:      method_descriptor

In [49]:
d = {'a': 1}

In [50]:
d.setdefault('a')

1

In [51]:
d.setdefault('b')

In [52]:
d

{'a': 1, 'b': None}

In [53]:
d.setdefault('x', []).append(1)

In [54]:
d

{'a': 1, 'b': None, 'x': [1]}

In [55]:
registry = {}

In [56]:
def register_at_call(name):
    """Register the decorated function at call time."""

    def _register(func):
        """Take the function."""

        @wraps(func)
        def __register(*args, **kwargs):
            """Take the function arguments."""
            registry.setdefault(name, []).append(func)
            return func(*args, **kwargs)

        return __register

    return _register


In [57]:
@register_at_call('add')
def add(a, b):
    return a + b

In [58]:
add(1, 2)


3

# Class Decorators

In [59]:
def mark(cls):
    cls.new = 100
    return cls

In [60]:
@mark
class A:
    pass

In [61]:
A.new

100

In [62]:
def assert_fluid(cls):
    assert 0 <= cls.temperature <= 100
    return cls

In [63]:
@assert_fluid
class Water:
    temperature = 25

In [64]:
Water.temperature

25

In [65]:
@assert_fluid
class Ice:
    temperature = -10

AssertionError: 

In [66]:
@assert_fluid
class Ice:
    temperature = 0

In [67]:
Ice.temperature

0

In [74]:
def check_name_length(max_len=30):
    """
    Check method name length.

    Raises a `NameError` if one method name of a decorated class is
    longer than `max_len`.
    """

    def _check_name_length(cls):
        for name, obj in cls.__dict__.items():
            if callable(obj) and len(name) > max_len:
                msg = (
                    f'name `{name}` too long\n'
                    f'found {len(name)} characters, '
                    f'only {max_len} are allowed'
                )
                raise NameError(msg)
        return cls

    return _check_name_length


In [78]:
@check_name_length()
class A:
    def method_with_a_very_long_name(self):
        pass

In [77]:
@check_name_length()
class B:
    def method_to_check_if_crossess_the_limit_and_will_see_if_passes(self):
        pass

NameError: name `method_to_check_if_crossess_the_limit_and_will_see_if_passes` too long
found 60 characters, only 30 are allowed